### Core libraries

In [ ]:
import random
import gc

---
---
---
---
---
---
---
---
---
---

### Mass of a DM particle from Illustris_3

In [ ]:
m_DM = 0.0338857141965349   # * 10^10 M_Sun

---
---
---
---
---

### Size of the grid

In [ ]:
ALL_sizes = [2**7, 2**8, 2**9, 2**9]   # 128 & 256 & 256_zone

### Numer of cores for multiprocessing

(Note: RAM is really the limiting factor.)

(Note: Below we offer the values used for an Apple M1 SIP with 16Gb of RAM and the computational time for size=512 | $\sigma$=0.6 | level=1$0^5$.)

In [ ]:
ALL_cores_no___1_cube                  = [4, 3, 3, 3] # (~2.5 min) ###
ALL_cores_no___2_Gaussian              = [4, 3, 3, 3] # (~0.5 min) ###
ALL_cores_no___3_Layer                 = [4, 3, 3, 3] # (~1 min) ###
ALL_cores_no___4_UOD_vals_and_lvls     = [4, 4, 2, 2] # (~1 min) ###
ALL_cores_no___5_Origins               = [4, 4, 3, 3] # (~0 min) #
ALL_cores_no___6_Levels_isolated_pairs = [4, 4, 3, 3] # (~0 min)
ALL_cores_no___7_Finder                = [4, 3, 3, 3] # (~0 min)

### Redshifts and their file number from Illustris3

In [ ]:
ALL_zPaths      = [[  "00"],
                   [  "00"],
                   [  "00",   "01",   "02",  "04",  "07",  "10",  "16",  "20",  "30",  "50",  "70",  "90",  "120",  "170"],
                   [  "00"]]
ALL_zPaths_nice = [[ "0.0"],
                   [ "0.0"],
                   [ "0.0",  "0.1",  "0.2", "0.4", "0.7", "1.0", "1.6", "2.0", "3.0", "5.0", "7.0", "9.0", "12.0", "17.0"],
                   [ "0.0"]]
ALL_zPaths_no   = [[   0.0],
                   [   0.0],
                   [   0.0,    0.1,    0.2,   0.4,   0.7,   1.0,   1.6,   2.0,   3.0,   5.0,   7.0,   9.0,   12.0,   17.0],
                   [   0.0]]
ALL_znos        = [[ "135"],
                   [ "135"],
                   [ "135",  "127",  "120", "108",  "95",  "85",  "73",  "68",  "60",  "49",  "41",  "35",   "28",   "21"],
                   [ "135"]]
ALL_ages        = [[13.752],
                   [13.752],
                   [13.752, 12.462, 11.353, 9.463, 7.411, 5.977, 4.120, 3.356, 2.195, 1.205, 0.782, 0.560,  0.376,  0.231],
                   [13.752]]

### Edges of the cubes we're interested in [cMpc/h]

(Note: 0-75 is the default for the whole cube.)

(Note: Since this project uses data cubes in the following steps, we only allow for cubic spapes.)

In [ ]:
ALL_edge_cube_cuts = [[[ 0,75],  [ 0,75],  [ 0,75]],
                      [[ 0,75],  [ 0,75],  [ 0,75]],
                      [[ 0,75],  [ 0,75],  [ 0,75]],
                      [[25,50],  [25,50],  [25,50]]]

In [ ]:
# We multiply by 1000 cecause the Illustris_3 data is in kpc and we onyl use those in the making of the cube.
ALL_eccs = [[[ecc_ij*1000 for ecc_ij in ecc_i] for ecc_i in ecc] for ecc in ALL_edge_cube_cuts]

### Grid size (dxyz)

(Note: These are in units of ckpc/h, since this is how the files were given.)

In [ ]:
ALL_d_xyzs = [(ecc[0][1]-ecc[0][0])/size for ecc, size in zip(ALL_eccs, ALL_sizes)]

---

### Paths

In [ ]:
# Define the directory where your notebook is located
notebook_directory = "./"

# Create a directory to store the executed notebooks
output_directory = "../output/"

# Path to save files to
basePath   = '..'
plots_path = basePath+"/Plots/"

ALL_file_paths = []
for z, size in enumerate(ALL_sizes):
    txt = basePath+"/Modified_Data_"+str(int(size))+('' if (ALL_edge_cube_cuts[z] == [[0,75], [0,75], [0,75]]) else '___'+txt_ZONE) +"/"
    txt_ZONE = "__".join("_".join(str(ALL_edge_cube_cuts[z][i][j]) for j in range(len(ALL_edge_cube_cuts[z][i]))) for i in range(len(ALL_edge_cube_cuts[z])))
    ALL_file_paths.append(txt)

---
---
---
---

### STD of the Gaussian smoothing in physical units [cMpc/h]

Those for all the current epoch and those for the it and all the previous ones (we use a fixed one for the previous frames, but if one desires different values, the code is easily adaptable).

In [ ]:
ALL_sigma_ps = [[[0.3, 0.4, 0.6, 1]],
                [[0.3, 0.4, 0.6, 1]],
                [[0.3, 0.4, 0.6, 1],
                 [          0.6   ],
                 [          0.6   ],
                 [          0.6   ],
                 [          0.6   ],
                 [          0.6   ],
                 [          0.6   ],
                 [          0.6   ],
                 [          0.6   ],
                 [          0.6   ],
                 [          0.6   ],
                 [          0.6   ],
                 [          0.6   ], 
                 [          0.6   ]],
                [[          0.6   ]]]

ALL_sigma_cs = []
for z, ALL_sigma_ps_i in enumerate(ALL_sigma_ps):
    d_xyz_sgm = ALL_d_xyzs[z]/1000
    ALL_sigma_cs.append([])
    for ALL_sigma_ps_ij in ALL_sigma_ps_i:
        ALL_sigma_cs[-1].append([])
        for ALL_sigma_ps_ijk in ALL_sigma_ps_ij:
            ALL_sigma_cs[-1][-1].append(round(ALL_sigma_ps_ijk/d_xyz_sgm, 3))  

### Number of levels to level the cubes

In [ ]:
ALL_lvls =         [[[[10**4], [10**4], [3*10**2, 10**3, 3*10**3, 10**4, 3*10**4, 10**5, 3*10**5       ], [10**4]]],
                    [[[10**5], [10**5], [3*10**2, 10**3, 3*10**3, 10**4, 3*10**4, 10**5, 3*10**5, 10**6], [10**5]]],
                    [[[10**5], [10**5], [3*10**2, 10**3, 3*10**3, 10**4, 3*10**4, 10**5, 3*10**5, 10**6], [10**5]],
                     [                  [                                         10**5                ]         ],
                     [                  [                                         10**5                ]         ],
                     [                  [                                         10**5                ]         ],
                     [                  [                                         10**5                ]         ],
                     [                  [                                         10**5                ]         ],
                     [                  [                                         10**5                ]         ],
                     [                  [                                         10**5                ]         ],
                     [                  [                                         10**5                ]         ],
                     [                  [                                         10**5                ]         ],
                     [                  [                                         10**5                ]         ],
                     [                  [                                         10**5                ]         ],
                     [                  [                                         10**5                ]         ],
                     [                  [                                         10**5                ]         ]],
                    [[                  [                                         10**5                ]         ]]]

### Under and Over densities

In [ ]:
ALL_Density_uods = [[[[[                       ]],  [[                       ]], [[], [], [], [                        [0.20,0.30]], [], [                                   ], []                                       ], [[                       ]]]],
                    [[[[                       ]],  [[                       ]], [[], [], [], [                                   ], [], [                        [0.20,0.30]], [], [                                   ]], [[                       ]]]],
                    [[[[[0.15,0.50],[0.20,0.30]]],  [[[0.15,0.50],[0.20,0.30]]], [[], [], [], [[0.00,1.00],[0.15,0.50],[0.20,0.30]], [], [[0.00,1.00],[0.15,0.50],[0.20,0.30]], [], [[0.00,1.00],[0.15,0.50],[0.20,0.30]]], [[[0.15,0.50],[0.20,0.30]]]],
                     [                                                           [                                                       [                        [0.20,0.30]]                                           ]                             ],
                     [                                                           [                                                       [                        [0.20,0.30]]                                           ]                             ],
                     [                                                           [                                                       [                        [0.20,0.30]]                                           ]                             ],
                     [                                                           [                                                       [            [0.15,0.50],[0.20,0.30]]                                           ]                             ],
                     [                                                           [                                                       [                        [0.20,0.30]]                                           ]                             ],
                     [                                                           [                                                       [                        [0.20,0.30]]                                           ]                             ],
                     [                                                           [                                                       [                        [0.20,0.30]]                                           ]                             ],
                     [                                                           [                                                       [            [0.15,0.50],[0.20,0.30]]                                           ]                             ],
                     [                                                           [                                                       [                        [0.20,0.30]]                                           ]                             ],
                     [                                                           [                                                       [                        [0.20,0.30]]                                           ]                             ],
                     [                                                           [                                                       [                        [0.20,0.30]]                                           ]                             ],
                     [                                                           [                                                       [            [0.15,0.50],[0.20,0.30]]                                           ]                             ],
                     [                                                           [                                                       [                        [0.20,0.30]]                                           ]                             ]],
                    [[                                                           [                                                       [[0.00,1.00],[0.15,0.50],[0.20,0.30]]                                           ]                             ]]] 

### Origins

In [ ]:
ALL_run_Origins =  [[[[True ], [True ], [True,    True,  True,    True,  True,    True,  True          ], [True  ]]],
                    [[[True ], [True ], [True,    True,  True,    True,  True,    True,  True,    True ], [True  ]]],
                    [[[True ], [True ], [True,    True,  True,    True,  True,    True,  True,    True ], [True  ]],
                     [                  [                                         True                 ]         ],
                     [                  [                                         True                 ]         ],
                     [                  [                                         True                 ]         ],
                     [                  [                                         True                 ]         ],
                     [                  [                                         True                 ]         ],
                     [                  [                                         True                 ]         ],
                     [                  [                                         True                 ]         ],
                     [                  [                                         True                 ]         ],
                     [                  [                                         True                 ]         ],
                     [                  [                                         True                 ]         ],
                     [                  [                                         True                 ]         ],
                     [                  [                                         True                 ]         ],
                     [                  [                                         True                 ]         ]],
                    [[                  [                                         True                 ]         ]]]

---

### MK1, MK2

In [ ]:
ALL_run_MKs      = [[[[[               ]],  [[               ]], [[], [], [], [                ["MK1","MK2"]], [], [                                   ], []                           ], [[               ]]]],
                    [[[[               ]],  [[               ]], [[], [], [], [                             ], [], [                      ["MK1","MK2"]], [], [                       ]], [[               ]]]],
                    [[[[["MK2"],["MK2"]]],  [[["MK2"],["MK2"]]], [[], [], [], [["MK2"],["MK2"],[      "MK2"]], [], [["MK1","MK2"],["MK2"],["MK1","MK2"]], [], [["MK2"],["MK2"],["MK2"]]], [[["MK2"],["MK2"]]]],
                     [                                           [                                                 [                      [      "MK2"]]                               ]                     ],
                     [                                           [                                                 [                      [      "MK2"]]                               ]                     ],
                     [                                           [                                                 [                      [      "MK2"]]                               ]                     ],
                     [                                           [                                                 [              ["MK2"],[      "MK2"]]                               ]                     ],
                     [                                           [                                                 [                      [      "MK2"]]                               ]                     ],
                     [                                           [                                                 [                      [      "MK2"]]                               ]                     ],
                     [                                           [                                                 [                      [      "MK2"]]                               ]                     ],
                     [                                           [                                                 [              ["MK2"],[      "MK2"]]                               ]                     ],
                     [                                           [                                                 [                      [      "MK2"]]                               ]                     ],
                     [                                           [                                                 [                      [      "MK2"]]                               ]                     ],
                     [                                           [                                                 [                      [      "MK2"]]                               ]                     ],
                     [                                           [                                                 [              ["MK2"],[      "MK2"]]                               ]                     ],
                     [                                           [                                                 [                      [      "MK2"]]                               ]                     ]],
                    [[                                           [                                                 [[      "MK2"],["MK2"],[      "MK2"]]                               ]                     ]]] 

---

### Save1%

In [ ]:
ALL_save1percs   = [[[[[               ]],  [[               ]], [[], [], [], [                [False,False]], [], [                                  ], []                           ], [[               ]]]],
                    [[[[               ]],  [[               ]], [[], [], [], [                             ], [], [                      [True, True]], [], [                       ]], [[               ]]]],
                    [[[[[False],[False]]],  [[[False],[False]]], [[], [], [], [[False],[False],[      False]], [], [[True ,True ],[False],[True, True]], [], [[False],[False],[False]]], [[[False],[False]]]],
                     [                                           [                                                 [                      [      True]]                               ]                     ],
                     [                                           [                                                 [                      [      True]]                               ]                     ],
                     [                                           [                                                 [                      [      True]]                               ]                     ],
                     [                                           [                                                 [              [False],[      True]]                               ]                     ],
                     [                                           [                                                 [                      [      True]]                               ]                     ],
                     [                                           [                                                 [                      [      True]]                               ]                     ],
                     [                                           [                                                 [                      [      True]]                               ]                     ],
                     [                                           [                                                 [              [False],[      True]]                               ]                     ],
                     [                                           [                                                 [                      [      True]]                               ]                     ],
                     [                                           [                                                 [                      [      True]]                               ]                     ],
                     [                                           [                                                 [                      [      True]]                               ]                     ],
                     [                                           [                                                 [              [False],[      True]]                               ]                     ],
                     [                                           [                                                 [                      [      True]]                               ]                     ]],
                    [[                                           [                                                 [[      True ],[True ],[      True]]                               ]                     ]]] 

---
---
---

### Number of permuations allowed in Finder

In [ ]:
times_we_tried_max = 2*3*4*5

### Random seed for replicating the results

As we later explain, even our code does contain parts where the seed definitely can play a crucial role.

In [ ]:
random.seed(9)

---
---
---